# Street Fighter 6 Match Deduction - Business Logic Documentation

This notebook documents the complete business logic, design decisions, and reasoning behind the `MatchDeductor` class and `deduct.py` script.

## Purpose
- **Specification notebook**: Documents business requirements and architectural decisions
- **Testing environment**: Load real export.json files and validate detection logic
- **Knowledge preservation**: Captures the reasoning process for future maintenance

---

## 1. Street Fighter 6 Match Structure Analysis

### Understanding the Game Mechanics

Before building the detection system, we analyzed how SF6 matches are structured:

```
TOURNAMENT MATCH
├── SET 1: Player A (RYU) vs Player B (CHUN-LI)
│   ├── Round 1: Timer 99→0, RYU wins
│   ├── Round 2: Timer 99→0, CHUN-LI wins  
│   └── Round 3: Timer 99→0, RYU wins → SET WINNER: RYU
│
├── SET 2: Player A (KEN) vs Player B (CAMMY)  [Character Switch]
│   ├── Round 1: Timer 99→0, CAMMY wins
│   └── Round 2: Timer 99→0, CAMMY wins → SET WINNER: CAMMY
│
└── SET 3: Player A (RYU) vs Player B (CHUN-LI)  [Back to original]
    ├── Round 1: Timer 99→0, CHUN-LI wins
    └── Round 2: Timer 99→0, CHUN-LI wins → SET WINNER: CHUN-LI
    
→ MATCH WINNER: Player B (2 sets vs 1 set)
```

### Key Business Rules Discovered

1. **Timer Logic**: Each round starts at 99 seconds and counts down to 0
2. **Set Definition**: Consecutive rounds with same character matchup
3. **Character Switches**: Players can change characters between sets
4. **Match Structure**: Either ≥2 sets OR 1 set with ≥3 rounds (special case)
5. **Transition Periods**: Camera cuts, replays, UI screens between rounds/sets

## 2. Detection Challenges & Solutions

### Challenge 1: OCR Gaps During Transitions
**Problem**: Between rounds, camera shows replays/analysis → no timer/character detection  
**Example**: LUKE vs DEE JAY match (00:14:00-00:19:15) was filtered out

**Solution**: Timer Pattern Coherency
- Don't rely on continuous detection
- Look for timer "reset" patterns (low value → high value)
- Allow gaps in detection as long as character context continues

### Challenge 2: Timer Start Value Variations
**Problem**: Rounds don't always start exactly at 99 due to detection timing  
**Example**: Timer detected at 97, 96, or 85 depending on when OCR catches it

**Solution**: Flexible Threshold System
- Original: Only accept timer starts ≥90
- Improved: Accept timer starts ≥80 with different confidence levels
- Detect significant jumps (>20 points) as potential round starts

### Challenge 3: Real Start Time Calculation
**Problem**: If timer detected at value X, when did the round actually start?  
**Example**: Timer shows 89 at 00:05:30 → round started when?

**Solution**: Reverse Timer Calculation
```python
# If detected timer=89 at 00:05:30
seconds_elapsed_since_99 = 99 - 89  # = 10 seconds
real_start_time = detection_time - elapsed - 1  # 00:05:30 - 10s - 1s = 00:05:19
```

## 3. Architecture Design Decisions

### Decision 1: Hierarchical Detection (Bottom-Up)
```
Raw Frames → Detect Rounds → Group into Sets → Group into Matches
```

**Reasoning**: 
- Rounds are the atomic unit (clear timer pattern)
- Sets emerge from character consistency
- Matches emerge from temporal grouping

### Decision 2: Validation at Each Level
```python
# Round validation: timer coverage + decreasing pattern + reasonable start value
# Set validation: minimum 2 rounds with same characters  
# Match validation: minimum 2 sets OR 1 set with 3+ rounds
```

**Reasoning**: SF6 tournament rules require these minimums for valid competition

### Decision 3: Timer Pattern Types
```python
# Type 1: Classic transition (low <50 → high ≥90)
# Type 2: Moderate jump (low <80 → medium ≥85) 
# Type 3: Significant jump (any → +20 points, if result ≥80)
```

**Reasoning**: Different scenarios need different detection sensitivity

## 4. Implementation Testing

Let's load the actual system and test it with real data:

In [ ]:
import sys
import os
import json
from datetime import datetime

# Add project root to path
sys.path.append('/home/nayte/Anagraph/replay-reader')

from src.match_deductor import MatchDeductor

print("✅ Imports successful")

### Load Real Export Data

In [ ]:
# Load the export.json file that was causing issues
export_file = '/home/nayte/Anagraph/replay-reader/output/EVODay2.export.json'

with open(export_file, 'r') as f:
    frames_data = json.load(f)

print(f"📊 Loaded {len(frames_data)} frames from {export_file}")
print(f"📅 Time range: {frames_data[0]['timestamp']} → {frames_data[-1]['timestamp']}")

# Show sample data structure
print("\n📋 Sample frame data:")
for i in [0, 100, 1000]:
    frame = frames_data[i]
    print(f"  {frame['timestamp']}: timer='{frame['timer_value']}', char1='{frame['character1']}', char2='{frame['character2']}'")

### Test Case: LUKE vs DEE JAY Detection

This was the problematic match that was filtered out before our improvements:

In [ ]:
# Find LUKE vs DEE JAY frames around 00:14:00
luke_deejay_frames = []
for frame in frames_data:
    if ('LUKE' in frame.get('character1', '') and 'DEE JAY' in frame.get('character2', '')) or \
       ('DEE JAY' in frame.get('character1', '') and 'LUKE' in frame.get('character2', '')):
        luke_deejay_frames.append(frame)

print(f"🎮 Found {len(luke_deejay_frames)} frames with LUKE vs DEE JAY")
if luke_deejay_frames:
    print(f"⏰ Time range: {luke_deejay_frames[0]['timestamp']} → {luke_deejay_frames[-1]['timestamp']}")
    
    # Show key frames with timer transitions
    print("\n🔍 Key frames with timer values:")
    for frame in luke_deejay_frames[:10]:
        if frame['timer_value']:
            print(f"  {frame['timestamp']}: timer={frame['timer_value']}, {frame['character1']} vs {frame['character2']}")

### Test Original Logic (Strict Detection)

In [ ]:
# Simulate original logic with strict timer requirements
class OriginalMatchDeductor(MatchDeductor):
    def _find_round_starts(self, parsed_frames):
        """Original logic: only accept timer ≥90 after low timer <50"""
        round_starts = []
        prev_timer = None
        
        for frame in parsed_frames:
            current_timer = frame['timer_value']
            
            if current_timer is not None and current_timer >= 90:
                if prev_timer is None or prev_timer < 50:
                    round_starts.append({
                        'timestamp': frame['timestamp'],
                        'timer_value': current_timer,
                        'prev_timer': prev_timer
                    })
            
            if current_timer is not None:
                prev_timer = current_timer
        
        return round_starts
    
    def _is_valid_round(self, round_data):
        """Original logic: require timer start ≥90"""
        timer_coverage = round_data['timer_coverage']
        timer_pattern = round_data['timer_pattern']
        start_timer = round_data['start_timer_value']
        
        coverage_ok = timer_coverage >= (1 - self.timer_tolerance)
        start_ok = start_timer >= 90  # Original strict requirement
        pattern_ok = timer_pattern['is_decreasing']
        
        return coverage_ok and start_ok and pattern_ok

# Test with original logic
original_deductor = OriginalMatchDeductor(debug=True)
original_results = original_deductor.analyze_frames(frames_data)

print(f"📊 Original Logic Results:")
print(f"   Matches: {original_results['stats']['total_matches_detected']}")
print(f"   Sets: {original_results['stats']['total_sets_detected']}")
print(f"   Rounds: {original_results['stats']['total_rounds_detected']}")

# Check if LUKE vs DEE JAY is detected
luke_deejay_found = False
for match in original_results['matches']:
    if ('LUKE' in match.get('player1', '') and 'DEE JAY' in match.get('player2', '')) or \
       ('DEE JAY' in match.get('player1', '') and 'LUKE' in match.get('player2', '')):
        luke_deejay_found = True
        print(f"✅ LUKE vs DEE JAY found in original logic")
        break

if not luke_deejay_found:
    print(f"❌ LUKE vs DEE JAY NOT found in original logic")

### Test Improved Logic (Flexible Detection)

In [ ]:
# Test with improved logic
improved_deductor = MatchDeductor(debug=True)
improved_results = improved_deductor.analyze_frames(frames_data)

print(f"📊 Improved Logic Results:")
print(f"   Matches: {improved_results['stats']['total_matches_detected']}")
print(f"   Sets: {improved_results['stats']['total_sets_detected']}")
print(f"   Rounds: {improved_results['stats']['total_rounds_detected']}")

# Check if LUKE vs DEE JAY is detected
luke_deejay_matches = []
for match in improved_results['matches']:
    if ('LUKE' in match.get('player1', '') and 'DEE JAY' in match.get('player2', '')) or \
       ('DEE JAY' in match.get('player1', '') and 'LUKE' in match.get('player2', '')):
        luke_deejay_matches.append(match)

if luke_deejay_matches:
    print(f"✅ LUKE vs DEE JAY found in improved logic!")
    for i, match in enumerate(luke_deejay_matches):
        print(f"   Match {i+1}: {match['start_time']}, {len(match['sets'])} sets")
        for set_data in match['sets']:
            print(f"     Set: {set_data['character1']} vs {set_data['character2']} ({set_data['rounds_count']} rounds)")
else:
    print(f"❌ LUKE vs DEE JAY still not found")

# Show improvement metrics
print(f"\n📈 Improvement Summary:")
print(f"   Matches: {original_results['stats']['total_matches_detected']} → {improved_results['stats']['total_matches_detected']} (+{improved_results['stats']['total_matches_detected'] - original_results['stats']['total_matches_detected']})")
print(f"   Rounds: {original_results['stats']['total_rounds_detected']} → {improved_results['stats']['total_rounds_detected']} (+{improved_results['stats']['total_rounds_detected'] - original_results['stats']['total_rounds_detected']})")

### Analyze Timer Pattern Detection

In [ ]:
# Analyze the specific timer patterns that were detected
deductor = MatchDeductor(debug=False)
parsed_frames = deductor._parse_and_validate_frames(frames_data)

# Find round starts with the improved logic
round_starts = deductor._find_round_starts(parsed_frames)

print(f"🔍 Timer Pattern Analysis:")
print(f"   Total round starts detected: {len(round_starts)}")

# Categorize by transition type
transition_types = {}
for start in round_starts:
    t_type = start.get('transition_type', 'unknown')
    if t_type not in transition_types:
        transition_types[t_type] = []
    transition_types[t_type].append(start)

for t_type, starts in transition_types.items():
    print(f"\n📊 {t_type.upper()} transitions: {len(starts)}")
    for start in starts[:3]:  # Show first 3 examples
        prev = start['prev_timer'] if start['prev_timer'] is not None else 'None'
        curr = start['timer_value']
        timestamp = start['timestamp'].strftime('%H:%M:%S')
        print(f"     {timestamp}: {prev} → {curr}")
    if len(starts) > 3:
        print(f"     ... and {len(starts) - 3} more")

## 5. Business Rules Validation

Verify that our detection respects SF6 tournament rules:

In [ ]:
# Validate business rules compliance
results = improved_results

print("🎯 Business Rules Validation:")
print("=" * 50)

# Rule 1: Sets must have ≥2 rounds
invalid_sets = []
for match in results['matches']:
    for set_data in match['sets']:
        if set_data['rounds_count'] < 2:
            invalid_sets.append(set_data)

print(f"📋 Rule 1 - Sets with ≥2 rounds:")
print(f"   Valid sets: {results['stats']['total_sets_detected'] - len(invalid_sets)}")
print(f"   Invalid sets: {len(invalid_sets)}")
if invalid_sets:
    print(f"   ❌ VIOLATION: Found sets with <2 rounds")
else:
    print(f"   ✅ PASSED: All sets have ≥2 rounds")

# Rule 2: Matches must have ≥2 sets OR 1 set with ≥3 rounds
invalid_matches = []
for match in results['matches']:
    sets_count = len(match['sets'])
    if sets_count >= 2:
        continue  # Valid: multiple sets
    elif sets_count == 1 and match['sets'][0]['rounds_count'] >= 3:
        continue  # Valid: single set with enough rounds
    else:
        invalid_matches.append(match)

print(f"\n📋 Rule 2 - Match structure validation:")
print(f"   Valid matches: {len(results['matches']) - len(invalid_matches)}")
print(f"   Invalid matches: {len(invalid_matches)}")
if invalid_matches:
    print(f"   ❌ VIOLATION: Found invalid match structures")
    for match in invalid_matches[:3]:
        print(f"     {match['start_time']}: {len(match['sets'])} sets, {match['sets'][0]['rounds_count']} rounds")
else:
    print(f"   ✅ PASSED: All matches have valid structure")

# Rule 3: Timer patterns should be decreasing within rounds
print(f"\n📋 Rule 3 - Timer patterns:")
print(f"   Timer detection rate: {results['stats']['timer_detection_rate']:.1%}")
print(f"   ✅ PASSED: Timer coherency logic implemented")

# Summary statistics
print(f"\n📊 Final Statistics:")
print(f"   Average sets per match: {results['stats']['avg_sets_per_match']:.1f}")
print(f"   Average rounds per set: {results['stats']['avg_rounds_per_set']:.1f}")
print(f"   Total tournament time: {frames_data[0]['timestamp']} → {frames_data[-1]['timestamp']}")

## 6. Edge Cases & Future Improvements

### Documented Edge Cases

1. **Very Short Rounds**: Some rounds last 5-15 seconds (quick victories)
2. **Timer Synchronization**: OCR detection timing vs actual game timer
3. **Character Recognition**: Similar character names (e.g., "M. BISON" vs "BISON")
4. **Tournament Formats**: Different set structures for different tournament stages

### Future Enhancement Ideas

1. **Winner Detection**: Analyze health bars or victory screens
2. **Player Name Recognition**: Extract player tags from overlays
3. **Round Win Conditions**: Distinguish timeout vs knockout
4. **Tournament Bracket Integration**: Match detected games to bracket structure

### Performance Considerations

- **Memory Usage**: Large export.json files (8000+ frames)
- **Processing Speed**: Timer pattern analysis is O(n) where n = frame count
- **Accuracy vs Speed**: More flexible detection = more computation

## 7. Conclusion & Design Summary

### Key Success Factors

1. **Domain Knowledge**: Understanding SF6 game mechanics was crucial
2. **Iterative Improvement**: Started strict, became flexible based on real data
3. **Hierarchical Validation**: Bottom-up approach (rounds → sets → matches)
4. **Pattern Recognition**: Timer coherency over rigid time-based rules

### Architecture Benefits

- **Modular Design**: Each detection level can be improved independently
- **Configurable Thresholds**: Easy to tune for different video qualities
- **Debug Visibility**: Comprehensive logging for troubleshooting
- **Statistical Validation**: Built-in metrics to assess detection quality

### Business Value

- **Automated Tournament Analysis**: Convert hours of video into structured data
- **Statistical Insights**: Match duration, character usage, tournament flow
- **Quality Assurance**: Detect incomplete or problematic tournament segments
- **Scalability**: Process multiple tournaments with consistent methodology

---

**This notebook serves as the definitive specification for the SF6 match detection system.**